# 태블로 RD 업데이트

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [9]:
class directory:
    dropbox_dir = 'C:/Users/MADUP/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
    
    raw_dir = dropbox_dir + ''
    # raw_data가 적재되어 있는 폴더 위치경로
    
    download_dir = 'C:/Users/MADUP/Downloads'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [10]:
import datetime

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    #today = datetime.date(year = 2022, month = 9, day = 1)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

rdate = report_date()

### 구글 스프레드 시트 읽기
- 특정 구글 스프레드 시트를 읽어오기 위해 활용하는 메소드 모음입니다.
- dcteam@madup-355605.iam.gserviceaccount.com 계정에 참조하고자 하는 구글 시트 공유가 필요합니다.

In [11]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import time


class spreadsheet:
    def spread_document_read(self, url):
        token_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/token'
        
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        json_file = token_dir + '/madup-355605-cd37b0ac201f.json'
        credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
        gc = gspread.authorize(credentials)
        read = gc.open_by_url(url)
        print('스프레드시트 읽기 완료')
        return read
    
    def spread_sheet(self, doc, sheet_name, col_num=0, row_num=0):
        while True :
            try :
                data_sheet = doc.worksheet(sheet_name)
                data_sheet_read = data_sheet.get_all_values()
                result = pd.DataFrame(data_sheet_read, columns=data_sheet_read[row_num]).iloc[row_num+1:, col_num:]
            except :
                print('API 오류로 15초 후 다시 시도 합니다.')
                time.sleep(15)
                continue
            break
        return result

spreadsheet = spreadsheet()

### 기타 설정
- 스프레드 시트 연동
- 태블로 설정 값 연동

In [12]:
class document():
    doc = spreadsheet.spread_document_read(
        'https://docs.google.com/spreadsheets/d/1efQrJpglhz0K9wKkl2QZ5VLrMu_hIUUnPG_uTSeI1Hk/edit#gid=2008544363')
    # 캠페인 정보 관리 시트 불러오기
    campaign_doc = spreadsheet.spread_document_read(
        'https://docs.google.com/spreadsheets/d/1w23AdEYLf4OJDlTElf2vwbVPttctSx-2nB51ZHQDEJQ/edit#gid=0')
    
class tableau_info():
    account_name = '쏘카'
    result_name = 'socar'

    raw_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/RD'
    asset_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/asset'
    result_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/result'

스프레드시트 읽기 완료
스프레드시트 읽기 완료


### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install [패키지명]`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [13]:
# 패키지 설치 코드 (패키지 설치 오류 발생시 아래 코드 실행해주세요.)
# !pip install pyarrow
# !pip install datetime

In [14]:
import warnings
warnings.filterwarnings('ignore')

# 자동화 코드 복사 붙여넣기

---

In [15]:
import pandas as pd

def tableau_custom_sheet(doc):
    # 태블로 대시보드 컬럼 설정 불러오기
    sheet_data = spreadsheet.spread_sheet(doc, tableau_info.account_name)
    return sheet_data

def get_column_dict(sheet_data):
    sheet_data_column = sheet_data.loc[sheet_data['원본 리포트 기준']!= '입력 금지', ['원본 리포트 기준', '태블로 RD']]
    sheet_data_column = sheet_data_column.loc[sheet_data_column['원본 리포트 기준']!='']
    column_dict = dict(zip(sheet_data_column['원본 리포트 기준'], sheet_data_column['태블로 RD']))
    return column_dict

def client_rd_read(column_dict, encoding = 'utf-8-sig'):
    use_cols = column_dict.keys()
    ## 광고주 리포트 데이터 가져오기
    raw_data = pd.read_csv(tableau_info.raw_dir + f'/{tableau_info.account_name}/{tableau_info.result_name}_{rdate.yearmonth}.csv',
                           usecols=use_cols, encoding=encoding)

    raw_data = raw_data.rename(columns=column_dict)
    raw_data = raw_data.loc[pd.to_datetime(raw_data['날짜']).dt.month == rdate.day_1.month]

    # string to numeric
    value_columns = ['도달', '노출', '클릭', '비용', '구매', '매출', '첫구매', '첫구매 매출', '설치', '재설치', '재실행', '가입']
    columns = raw_data.columns

    for col in value_columns:
        if col in columns:
            raw_data[col] = raw_data[col].fillna(0)
            if raw_data[col].dtypes == 'object':
                raw_data[col] = raw_data[col].str.strip()
                raw_data[col] = raw_data[col].str.replace('-', '0')
                raw_data[col] = raw_data[col].str.replace(',', '')
                raw_data[col] = pd.to_numeric(raw_data[col])

    return raw_data

def campaign_info_sheet(campaign_doc):

    campaign_sheet = spreadsheet.spread_sheet(campaign_doc, '캠페인 정보')
    campaign_sheet = campaign_sheet.loc[campaign_sheet['광고주'] == tableau_info.account_name]
    campaign_sheet = campaign_sheet.loc[campaign_sheet['매체'].isin(['페이스북', '카카오', '구글'])]
    campaign_sheet = campaign_sheet[
        ['매체', '광고 상품', '캠페인', '광고그룹', '최적화 엔진', '캠페인 구분', '오디언스', 'OS', 'KPI', '캠페인 라벨']]
    campaign_sheet = campaign_sheet.drop_duplicates(['매체', '캠페인', '광고그룹'], keep='last')
    return campaign_sheet

    # 컬럼명 기준 딕셔너리 생성

def data_exception(raw_data, doc):
    # 매체별 필터 기준
    facebook_list = list(sheet_data.loc[sheet_data['페이스북 필터'].str.len() > 0, '페이스북 필터'])
    kakao_list = list(sheet_data.loc[sheet_data['카카오 필터'].str.len() > 0, '카카오 필터'])
    google_list = list(sheet_data.loc[sheet_data['구글 필터'].str.len() > 0, '구글 필터'])

    if len(facebook_list) != 0 :
        raw_data.loc[raw_data['매체'].isin(facebook_list), '매체'] = '페이스북'
    if len(kakao_list) != 0 :
        raw_data.loc[raw_data['매체'].isin(kakao_list), '매체'] = '카카오'
    if len(google_list) != 0:
        raw_data.loc[raw_data['매체'].isin(google_list), '매체'] = '구글'

    # 예외처리 정보 불러오기
    exception_sheet = spreadsheet.spread_sheet(doc, '예외처리')
    exception_sheet = exception_sheet.loc[(exception_sheet['체크박스']=='TRUE')&(exception_sheet['광고주']==tableau_info.account_name)]
    exception_sheet.index = range(0, len(exception_sheet))

    for i in exception_sheet.index:
        func = exception_sheet.iloc[i]['방식']
        col = exception_sheet.iloc[i]['구분']
        media = exception_sheet.iloc[i]['매체']
        name = exception_sheet.iloc[i]['이름']
        alt = exception_sheet.iloc[i]['대체 값']

        if func == 'EXCLUDE':
            raw_data = raw_data.loc[~((raw_data['매체']==media) & (raw_data[col]==name))]
        elif func == 'REPLACE' :
            raw_data.loc[((raw_data['매체'] == media) & (raw_data[col] == name)), name] = alt

    #creative_raw = creative_raw.loc[creative_raw['매체'].isin(['페이스북', '카카오', '구글'])]
    raw_data.loc[raw_data['매체']=='구글', '소재'] = raw_data['광고그룹']
    return raw_data

def asset_data_read(campaign_doc):
    # 애셋 정보 불러오기
    account_sheet = spreadsheet.spread_sheet(campaign_doc, 'Owner 정보')
    account_sheet = account_sheet.rename(columns = {'Owner ID' : 'owner_id'})
    account_sheet = account_sheet.drop_duplicates('owner_id')
    asset_data = pd.read_csv(tableau_info.asset_dir + f'/total_asset_data_{rdate.yearmonth}.csv')
    asset_data_merge = asset_data.merge(account_sheet, on ='owner_id', how='left')
    asset_data_merge = asset_data_merge.loc[asset_data_merge['광고주']==tableau_info.account_name]
    del asset_data_merge['owner_id'], asset_data_merge['광고주']
    return asset_data_merge


In [16]:
# 태블로 커스텀 정보 시트 불러오기
sheet_data = tableau_custom_sheet(document.doc)

# 원본 데이터 컬럼 <-> 태블로 템플릿용 데이터 컬럼 딕셔너리
column_dict = get_column_dict(sheet_data)

# 드롭박스에 적재된 광고주 리포트 RD 불러오기
raw_data = client_rd_read(column_dict, 'cp949')

# 캠페인 정보 시트 불러오기
campaign_sheet = campaign_info_sheet(document.campaign_doc)

# 리포트 RD 캠페인 정보 merge
raw_data = raw_data.merge(campaign_sheet, on=['매체', '캠페인', '광고그룹'], how='left')

# 데이터 예외처리
except_raw = data_exception(raw_data, document.doc)

# 소재 데이터 정보 불러오기
asset_data_merge = asset_data_read(document.campaign_doc)

# 리포트 RD <-> 소재 정보 merge
raw_merge_final = except_raw.merge(asset_data_merge, on = ['매체', '캠페인', '광고그룹', '소재'], how = 'left')

# 드롭박스 저장
raw_merge_final.to_csv(tableau_info.result_dir + f'/{tableau_info.account_name}/tableau_creative_rd_{tableau_info.result_name}_{rdate.yearmonth}.csv', index=False, encoding='utf-8-sig')